# stocks 정보 최신화

In [1]:
# pykrx to mysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from openpyxl import load_workbook
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

start = '20200101'
end = '20211025'
# end = str(datetime.now().date()).replace('-', '')
# period = pd.date_range(start=start, end=end).strftime("%Y%m%d").tolist()
markets = ['KOSPI', 'KOSDAQ']

# 1. 이관 시 ticker기준 조회
for market in markets:
  tickers = stock.get_market_ticker_list(market = market)
  for ticker in tickers:
    try:
      df = stock.get_market_ohlcv_by_date(start, end, ticker)
      time.sleep(0.5)
      df['ticker'] = ticker
      df = df.rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})    
      df.index.names = ['date']
      df = df[['ticker', 'open', 'high', 'low', 'close', 'volume']]

      db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
      conn = db_connection.connect()
      df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
      conn.close()
    except Exception as ex:
      print(ex, ticker)
      pass

# 2. 업데이트 시 date기준 조회
# for date in period:
#   for market in markets:
#     try:
#       db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
#       conn = db_connection.connect()
#       df = stock.get_market_ohlcv_by_ticker(date, market=market)
#       time.sleep(0.5)
#       df['date'] = date
#       df = df.reset_index().set_index('date').drop(['거래대금', '등락률'], axis=1).rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})
#       df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
#       conn.close()
#     except Exception as ex:
#       print(ex, date)
#       pass
# 
# 최신화 후 DELETE FROM stocks_price WHERE volume = 0; 실행
# 

# 볼린저밴드 구하기

In [3]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      # print(ticker[0], date[0].strftime("%Y%m%d"))
      try:
        getLast20InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit 20"
        cursor.execute(getLast20InfoSql)
        last20InfoDf = pd.DataFrame(cursor.fetchall())
        if len(last20InfoDf.index) < 20:
          continue
        if last20InfoDf.iloc[0][6] == 0:
          continue
        
        last20InfoDfClose = last20InfoDf.loc[:, 5]
        avg = last20InfoDfClose.mean()
        std = last20InfoDfClose.std()
        tmpInfo = last20InfoDf.loc[0, [0, 1, 5]]
        hi = round(avg + std * 2)
        me = round(avg)
        lo = round(avg - std * 2)
        bw = (hi - lo) / me
        pos = (tmpInfo[5] - lo) / (hi - lo)

        insertBollSql = "REPLACE INTO boll(date, ticker, close, low, medium, high, bandWidth, position) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)" 
        cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw, pos))
        db.commit()
      except Exception as ex1:
        print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"), tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw, pos)
        # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

C:\Users\Jack\AppData\Local\Temp/ipykernel_8144/3470133148.py:43: RuntimeWarning: invalid value encountered in longlong_scalars
  pos = (tmpInfo[5] - lo) / (hi - lo)


ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`boll`.`position` at row 1") 083380 20190318 2019-03-18 083380 5010 5010 5010 5010 0.0 nan


# Enhancing bollinger

In [38]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'
db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [20]

start = '20190101'
end = '20211025'

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  for ticker in tickers:
    for day in n_days:
      try:
        # Get ticker's price
        cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
        priceDf = pd.DataFrame(cursor.fetchall())
        priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'}).drop(['open', 'high', 'low', 'volume'], axis=1)
        # 
        # 실행 전 Alter table로 period 컬럼 추가할 것!!
        # 
        priceDf['period'] = day
        priceDf['low'] = 0
        priceDf['medium'] = 0
        priceDf['high'] = 0
        priceDf['bandWidth'] = 0.0
        priceDf['position'] = 0.0
        # Set bandWidth, Bollinger
        for idx in range(0, len(priceDf)):
          if idx + day > len(priceDf) - 1:
            continue
          copyDf = priceDf[idx:idx+day].loc[:, 'close'].copy()
          avg = copyDf.mean()
          std = copyDf.std()
          
          lo = round(avg - std * 2)
          me = round(avg)
          hi = round(avg + std * 2)
          bw = (hi - lo) / me
          pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)
          priceDf.at[idx, 'low'] = lo
          priceDf.at[idx, 'medium'] = me
          priceDf.at[idx, 'high'] = hi
          priceDf.at[idx, 'bandWidth'] = bw
          priceDf.at[idx, 'position'] = pos

        bollDf = priceDf.reset_index().set_index('date').drop(['index'], axis=1)
        
        db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
        conn = db_connection.connect()
        bollDf.to_sql(name='boll', con=db_connection, if_exists='append', index=True, index_label="date")        
        conn.close()
      except Exception as ex1:
        print('ex1', ex1)
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\Jack\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pymysql\connections.py", line 756, in _write_bytes
    self._sock.sendall(data)
ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jack\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sqlalchemy\pool\base.py", line 682, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\Jack\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sqlalchemy\pool\base.py", line 887, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\Jack\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sqlalchemy\engine\default.py", line 667, in do_rollback
    dbapi_connection.rollback()
  File "C:\Users\Jack\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pymysql\connections.p

ex1 (pymysql.err.IntegrityError) (1062, "Duplicate entry '000020-2021-10-25-20' for key 'PRIMARY'")
[SQL: INSERT INTO boll (date, ticker, close, period, low, medium, high, `bandWidth`, position) VALUES (%(date)s, %(ticker)s, %(close)s, %(period)s, %(low)s, %(medium)s, %(high)s, %(bandWidth)s, %(position)s)]
[parameters: ({'date': datetime.date(2021, 10, 25), 'ticker': '000020', 'close': 16150, 'period': 20, 'low': 15714, 'medium': 17915, 'high': 20116, 'bandWidth': 0.24571588054702764, 'position': 0.09904588823262153}, {'date': datetime.date(2021, 10, 22), 'ticker': '000020', 'close': 16650, 'period': 20, 'low': 15957, 'medium': 17998, 'high': 20038, 'bandWidth': 0.22674741637959772, 'position': 0.16981132075471697}, {'date': datetime.date(2021, 10, 21), 'ticker': '000020', 'close': 16350, 'period': 20, 'low': 15555, 'medium': 17918, 'high': 20280, 'bandWidth': 0.2637013059493247, 'position': 0.16825396825396827}, {'date': datetime.date(2021, 10, 20), 'ticker': '000020', 'close': 17400

C:\Users\Jack\AppData\Local\Temp/ipykernel_18640/2687744687.py:52: RuntimeWarning: invalid value encountered in longlong_scalars
  pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)


# Z-SCORE
- (현재값 - 평균) / 표준편차

In [127]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

# n_days = [10, 20, 30, 90, 180]
n_days = [180]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        try:
          getLastNBollSql = "SELECT * FROM boll WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day)
          cursor.execute(getLastNBollSql)
          lastNBollDf = pd.DataFrame(cursor.fetchall())
          if len(lastNBollDf.index) < day:
            continue

          lastNBollBandWidthDf = lastNBollDf.loc[:, 6]
          avg = lastNBollBandWidthDf.mean()
          std = lastNBollBandWidthDf.std()
          tmpInfo = lastNBollDf.loc[0, [0, 1, 6]]
          z_score = (tmpInfo[6] - avg) / std
          
          insertZScoreSql = "REPLACE INTO z_score(date, ticker, period, bandWidth, z_score) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertZScoreSql, (tmpInfo[0], tmpInfo[1], day, tmpInfo[6], z_score))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# Enhancing Z-score

In [ ]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

# n_days = [10, 20, 30, 90, 180]
n_days = [180]

start = '20190101'
end = '20211025'

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  for ticker in tickers:
    for day in n_days:
      try:
        # Get ticker's bollinger
        cursor.execute("SELECT * FROM boll WHERE ticker = '" + ticker[0] + " AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
        bollDf = pd.DataFrame(cursor.fetchall())
        bollDf = bollDf.rename(columns={0: 'date', 1:'ticker', 2:'close', 3:'low', 4:'medium', 5:'high', 6:'bandWidth', 7:'position'})
        bollDf['period'] = day
        bollDf['z_score'] = 0
        # Set z_score
        for idx in range(0, len(bollDf)):
          if idx + day > len(bollDf) - 1:
            continue
          copyDf = bollDf[idx:idx+day].loc[:, 'bandWidth'].copy()
          avg = copyDf.mean()
          std = copyDf.std()
          bollDf.at[idx, 'z_score'] = (bollDf.iloc[idx]['bandWidth'] - avg) / std
        bollDf = bollDf.reset_index().set_index('date').drop(['index', 'close', 'low', 'medium', 'high', 'posision'], axis=1)
        # db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
        # conn = db_connection.connect()
        # bollDf.to_sql(name='z_score', con=db_connection, if_exists='append', index=True, index_label="date")        
        # conn.close()
      except Exception as ex1:
        print('ex1', ex1, ticker[0])
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# MFI
- TP(Typical price) = (당일 고가 + 당일 저가 + 당일 종가) / 3
- RMF = TP * 당일 거래량
  - Positive RMF: 당일 TP > 전일 TP
  - Negative RMF: 당일 TP < 전일 TP
- MFI = n일간 positive RMF 합계 / (n일간 positive RMF 합계 + n일간 negative RMF 합계) * 100

In [ ]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        try:
          getLast10InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day + 1)
          cursor.execute(getLast10InfoSql)
          last10InfoDf = pd.DataFrame(cursor.fetchall())
          if len(last10InfoDf.index) < day + 1:
            continue
          tpList = []
          for idx in last10InfoDf.index:
            info = last10InfoDf.loc[idx]
            tpList.append(round((info[3] + info[4] + info[5]) / 3))
          last10InfoDf['TP'] = tpList

          positiveRMF = 0
          negativeRMF = 0
          for idx in range(0, len(last10InfoDf) - 1):
            today = last10InfoDf.iloc[idx]
            yesterday = last10InfoDf.iloc[idx + 1]
            if today['TP'] > yesterday['TP']:
              # positive
              positiveRMF += today['TP'] * today[6]
            elif today['TP'] < yesterday['TP']:
              # negative
              negativeRMF += today['TP'] * today[6]
          
          MFI = positiveRMF / (positiveRMF + negativeRMF) * 100
          todayInfo = last10InfoDf.iloc[0, [0, 1, 7]]

          insertBollSql = "REPLACE INTO mfi(date, ticker, period, tp, mfi) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertBollSql, (todayInfo[0], todayInfo[1], day, todayInfo['TP'], MFI))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# Enhancing MFI

In [61]:
import pandas as pd
import pymysql
import datetime
from sqlalchemy import create_engine

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10, 14]

start = '20190101'
end = '20211025'

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  for ticker in tickers:
    for day in n_days:
      try:
        # Get ticker's price
        cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date DESC")
        priceDf = pd.DataFrame(cursor.fetchall())
        priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
        priceDf['period'] = day
        priceDf['tp'] = 0
        priceDf['mfi'] = 0
        priceDf['mfi_diff'] = 0
        # Set typical price
        for idx in range(0, len(priceDf)):
          row = priceDf.iloc[idx]
          priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
        # Set MFI
        for idx in range(0, len(priceDf)):
          if idx + day > len(priceDf) - 1:
            continue
          positiveRMF = 0
          negativeRMF = 0          
          for i in range(idx, idx + day):
            today = priceDf.iloc[i]
            yesterday = priceDf.iloc[i + 1]
            if today['tp'] > yesterday['tp']:
              positiveRMF += today['tp'] * today['volume']
            elif today['tp'] < yesterday['tp']:
              negativeRMF += today['tp'] * today['volume']
          MFI = round(positiveRMF / (positiveRMF + negativeRMF) * 100)
          priceDf.at[idx, 'mfi'] = MFI
          if idx > 0:
            priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
        mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
        db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
        conn = db_connection.connect()
        mfiDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
        conn.close()
      except Exception as ex1:
        print('ex1', ex1, ticker[0])
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

ex1 division by zero 000040
ex1 division by zero 000150
ex1 division by zero 000155
ex1 division by zero 000157
ex1 division by zero 000210
ex1 division by zero 000210
ex1 division by zero 000215
ex1 division by zero 000215
ex1 division by zero 001020


C:\Users\Jack\AppData\Local\Temp/ipykernel_15928/2546899101.py:54: RuntimeWarning: invalid value encountered in longlong_scalars
  MFI = round(positiveRMF / (positiveRMF + negativeRMF) * 100)


ex1 cannot convert float NaN to integer 001020
ex1 division by zero 001210
ex1 division by zero 001770
ex1 division by zero 001770
ex1 division by zero 002070
ex1 division by zero 002380
ex1 division by zero 002380
ex1 division by zero 002420
ex1 division by zero 002420
ex1 division by zero 002630
ex1 division by zero 002840
ex1 division by zero 003280
ex1 division by zero 003280
ex1 division by zero 003410
ex1 division by zero 003410
ex1 division by zero 003550
ex1 division by zero 003550
ex1 division by zero 003555
ex1 division by zero 003555
ex1 division by zero 003620
ex1 division by zero 003620
ex1 division by zero 004150
ex1 cannot convert float NaN to integer 004150
ex1 division by zero 004540
ex1 division by zero 004540
ex1 division by zero 004545
ex1 division by zero 004545
ex1 division by zero 004920
ex1 cannot convert float NaN to integer 004920
ex1 cannot convert float NaN to integer 005420
ex1 division by zero 005850
ex1 division by zero 005850
ex1 division by zero 006580


# bollinger squeeze back test

In [27]:
import pandas as pd
import pymysql
import datetime
import traceback
import itertools

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### Back test 파라미터
## 기간
start = "20210101"
end = "20211001"
## 투자금
amount = 1000000

param_grid = {
              'market': ['KOSPI'],
              'position_bid': [0.6],
              'position_ask': [0.3],
              'mfi_bid': [75],
              'mfi_ask': [10],
              'z_score': [-1.2],
              'maxPnl': [20],
              'z_score_period': [180],
              'mfi_period': [10]
              }
# generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pnlHistory = []

for params in all_params:
  market = params['market']
  position_bid = params['position_bid']
  position_ask = params['position_ask']
  mfi_bid = params['mfi_bid']
  mfi_ask = params['mfi_ask']
  z_score = params['z_score']
  maxPnl = params['maxPnl']
  z_score_period = params['z_score_period']
  mfi_period = params['mfi_period']

  try:
    if market != 'ALL':
      cursor.execute("SELECT DISTINCT ticker FROM stocks_info WHERE market='" + market + "'")
    else:
      cursor.execute("SELECT DISTINCT ticker FROM stocks_info")
    tickers = list(cursor.fetchall())

    totalHistory = []
    orderHistory = []
    totalPNL = 0
    winCnt, loseCnt, winRate = 0, 0, 0
    for ticker in tickers:
      try:
        # Get bollinger band dataframe
        cursor.execute("SELECT * FROM boll WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' ORDER BY date")
        bollDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'close', 3:'low', 4:'medium', 5:'high', 6:'bandWidth', 7:'position'})
        bollDf.set_index('date', drop=True, inplace=True)
        # Get MFI dataframe
        cursor.execute("SELECT * FROM mfi WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' AND period = '" + str(mfi_period) + "' ORDER BY date")
        mfiDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'tp', 4:'mfi'})      
        mfiDf.set_index('date', drop=True, inplace=True)
        mfiDf = mfiDf.drop(['ticker', 'period'], axis=1)
        # Get z-score dataframe
        cursor.execute("SELECT * FROM z_score WHERE ticker = '" + ticker[0] + "' AND date >= '"+ start +"' AND date <= '"+ end + "' AND period = '" + str(z_score_period) + "' ORDER BY date")
        zscoreDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'bandWidth', 4:'z_score'})      
        zscoreDf = zscoreDf.drop(['ticker', 'period', 'bandWidth'], axis=1)
        zscoreDf.set_index('date', drop=True, inplace=True)
        sumDf = pd.concat([bollDf, mfiDf, zscoreDf], axis=1)

        isBid = False
        bidPrice = 0
        bidDate = '000000'      
        for idx in sumDf.index:
          row = sumDf.loc[idx].copy()
          if pd.isna(row['z_score']):
            continue
          # Bid Step
          if not isBid:
            if row['position'] >= position_bid and row['mfi'] >= mfi_bid and row['z_score'] < z_score:
              row['status'] = 'bid'
              bidPrice = row['close']
              bidDate = row.name.strftime("%Y%m%d")
              totalHistory.append(row)
              orderHistory.append(['bid', ticker[0], bidDate, bidPrice])
              isBid = True
          # Ask Step
          else:
            askPrice = row['close']
            askDate = row.name.strftime("%Y%m%d")
            pnl = (1 - (bidPrice / askPrice)) * 100
            if row['position'] <= position_ask or row['mfi'] <= mfi_ask or pnl > maxPnl:
              row['status'] = 'ask'
              totalHistory.append(row)
              orderHistory.append(['ask', ticker[0], askDate, askPrice])
              isBid = False
              if pnl > 0:
                winCnt += 1
              else:
                loseCnt += 1
              if not pd.isna(pnl):
                totalPNL += pnl
      except Exception as ex1:
        # print('ex1', traceback.format_exc(), ex1, ticker[0])
        print('ex1',  ex1, ticker[0])
        pass
    winRate = (winCnt / (winCnt + loseCnt)) * 100
    pnlHistory.append([params, winRate, totalPNL, totalHistory, orderHistory])    
  except Exception as ex2:
    print('ex2', ex2)
    pass

db.close()

ex1 "None of ['date'] are in the columns" 000040
ex1 "None of ['date'] are in the columns" 000150
ex1 "None of ['date'] are in the columns" 000155
ex1 "None of ['date'] are in the columns" 000157
ex1 "None of ['date'] are in the columns" 000210
ex1 "None of ['date'] are in the columns" 000215
ex1 "None of ['date'] are in the columns" 001020
ex1 "None of ['date'] are in the columns" 001210
ex1 "None of ['date'] are in the columns" 001770
ex1 "None of ['date'] are in the columns" 002070
ex1 "None of ['date'] are in the columns" 002380
ex1 "None of ['date'] are in the columns" 002420
ex1 "None of ['date'] are in the columns" 002630
ex1 "None of ['date'] are in the columns" 002840
ex1 "None of ['date'] are in the columns" 003280
ex1 "None of ['date'] are in the columns" 003410
ex1 "None of ['date'] are in the columns" 003550
ex1 "None of ['date'] are in the columns" 003555
ex1 "None of ['date'] are in the columns" 003620
ex1 "None of ['date'] are in the columns" 004150
ex1 "None of ['date'

In [28]:
# pnlHistory.sort(key= lambda x: [-x[2], -x[1]])
for idx, history in enumerate(pnlHistory):
  print(history[0], history[1], history[2])
  # print(markets[idx], history[0], history[1])

{'market': 'KOSPI', 'position_bid': 0.6, 'position_ask': 0.3, 'mfi_bid': 75, 'mfi_ask': 10, 'z_score': -1.2, 'maxPnl': 20, 'z_score_period': 180, 'mfi_period': 10} 41.55844155844156 684.6652375964878


In [29]:
orders = pnlHistory[0][4]
orders.sort(key = lambda x : x[2])
# orders
cnt = 0
maxCnt = 0
maxCntDate = '000000'
for order in orders:
  if order[0] == 'bid':
    cnt += 1
  else :
    cnt -= 1
  if cnt > maxCnt:
    maxCnt = cnt
    maxCntDate = order[2]
print(maxCntDate, maxCnt)

# for order  in orders:
  # print(order)

20210426 59


# Bollinger Backtest (date)

In [46]:
import pandas as pd
import pymysql
import datetime
import traceback
import itertools

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### Back test 파라미터
## 기간
start = "20210701"
end = "20211108"
## 투자금
totalAmount = 10000000
step = 10
tradeAmount = totalAmount / step

param_grid = {
              'market': ['KOSPI'],
              'position_bid': [0.1],
              'position_ask': [0.8],
              'mfi_bid': [10],
              'mfi_ask': [90],
              'boll_period': [20],
              'mfi_period': [10],
              'minCap': [1000000000000],
              # 'z_score': [-1.5],
              # 'maxPnl': [10],
              # 'lossCutPnl': [-20],
              # 'z_score_period': [180]
              }
# generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

tradeHistorys = []

for params in all_params:
  market = params['market']
  position_bid = params['position_bid']
  position_ask = params['position_ask']
  mfi_bid = params['mfi_bid']
  mfi_ask = params['mfi_ask']
  boll_period = params['boll_period']
  mfi_period = params['mfi_period']
  minCap = params['minCap']
  # lossCutPnl = params['lossCutPnl']

  try:
    # Ticker, Period init
    getPeriodSql = "SELECT DISTINCT date FROM stocks_price WHERE date >= "+ start +" AND date <= "+ end +" ORDER BY date"
    cursor.execute(getPeriodSql)
    periods = list(map(lambda p: p[0].strftime("%Y%m%d"), cursor.fetchall()))

    bidBasket = {}
    tradeHistory = []

    for date in periods:
      try:
        sql = """
        SELECT BOLMFI.*, INFO.name, INFO.market, INFO.cap
        FROM (
          SELECT BOL.*, MFI.tp, MFI.mfi, MFI.mfi_diff
          FROM (
            SELECT * 
            FROM boll 
            WHERE date = %s
            AND period = %s
          ) AS BOL
          JOIN (
            SELECT ticker, tp, mfi, mfi_diff
            FROM mfi
            WHERE date = %s
            AND period = %s
          ) AS MFI
          ON (BOL.ticker = MFI.ticker)
        ) AS BOLMFI
        JOIN (
          SELECT ticker, name, market, cap
          FROM stocks_info
          WHERE cap > %s
          AND market = '%s'
        ) AS INFO
        ON (BOLMFI.ticker = INFO.ticker)
        ORDER BY INFO.cap DESC
        """ % (date, boll_period, date, mfi_period, minCap, market)
        # """ % (date, boll_period, date, mfi_period, minCap, market)
        cursor.execute(sql)
        sumDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'close', 4:'low', 5:'medium', 6:'high', 7:'bandWidth', 8:'position', 9: 'tp', 10: 'mfi', 11: 'mfi_diff', 12: 'name', 13: 'market', 14:'cap'})
        sumDf.set_index('ticker', drop=True, inplace=True)

        for idx in sumDf.index:
          row = sumDf.loc[idx].copy()
          ticker = row.name
          ## BID
          ### 1. Bollinger
          if row['position'] <= position_bid and row['mfi'] <= mfi_bid and row['mfi'] > 0 and row['mfi_diff'] > 0:
          ### 2. 급등주 잡기
          # if row['position'] >= position_bid and row['mfi'] >= mfi_bid and row['z_score'] < z_score:       
          # if row['position'] >= 0.8 and row['mfi'] >= 85 and row['mfi'] - row['mfi_diff']  < 85:
            # 이미 매수한 종목 확인
            if ticker in bidBasket:
              continue
            # 매수
            elif ticker not in bidBasket and len(bidBasket) < step:
              bidBasket[ticker] = [row['date'], ticker, row['close']]
              tradeHistory.append(['bid', row['date'], ticker, row['close'], row['position'], row['mfi'], row['cap']])
          if ticker in bidBasket:
            bidPrice = bidBasket[ticker][2]
            pnl = (1 - (bidPrice / row['close'])) * 100
            ## ASK
            ### 1. Bollinger
            if row['position'] >= position_ask or row['mfi'] >= mfi_ask:            # 매수종목여부 확인
            # if row['position'] >= position_ask or row['mfi'] >= mfi_ask or pnl < lossCutPnl:            # 매수종목여부 확인
            ### 2. 급등주 잡기
            # if row['position'] <= position_ask or row['mfi'] <= mfi_ask or pnl > maxPnl:
            # if row['position'] <= 0.2 or row['mfi'] <= 20 or pnl > 5 :
              del(bidBasket[ticker])
              tradeHistory.append(['ask', row['date'], ticker, row['close'], row['position'], row['mfi'], row['cap'], pnl, tradeAmount * pnl * 0.01])
      except Exception as ex1:
        print('ex1', traceback.format_exc(), ex1)
        # print('ex1', traceback.format_exc(), ex1, ticker[0])
    tradeHistorys.append([params, tradeHistory])
  except Exception as ex2:
    print('ex2', ex2)
    pass

db.close()

In [47]:
answer = []

for params, tradeHistory in tradeHistorys:
  amt = 0
  pnl = 0
  w, l = 0, 0
  for trade in tradeHistory:
    if trade[0] == 'ask':
      pnl += trade[7]
      amt += trade[8]
      if trade[7] > 0:
        w += 1
      else:
        l += 1
  tmp = tradeHistory
  tmp.sort(key = lambda x: x[1])

  cnt = 0
  maxCnt = 0
  for t in tmp:
    if t[0] == 'bid':
      cnt += 1
    else:
      cnt -= 1
    if cnt > maxCnt:
      maxCnt = cnt

  if w + l != 0:
    answer.append([params, amt, w / (w+l), maxCnt])
    # print(params)
    # print(pnl, amt, w / (w + l))

answer.sort(key=lambda x: [-x[1], -x[2]])
for trade in answer:
  print(trade)

[{'market': 'KOSPI', 'position_bid': 0.1, 'position_ask': 0.8, 'mfi_bid': 10, 'mfi_ask': 90, 'boll_period': 20, 'mfi_period': 10, 'minCap': 1000000000000}, 606062.3287122858, 0.7692307692307693, 10]


In [48]:
for trade in sorted(tradeHistorys[0][1], key = lambda x: [x[1]]):
# for trade in tradeHistory:
  print(trade)

# for trade in tradeHistory:
#   if trade[0] == 'ask' and trade[7] < 0:
#     print(trade)

['bid', datetime.date(2021, 7, 30), '192820', 124000, 0.04672062224090814, 8.377117311417539, 1373290589000]
['bid', datetime.date(2021, 8, 3), '000150', 93000, 0.07186147186147186, 6.915673993119845, 1536716655000]
['bid', datetime.date(2021, 8, 11), '004000', 69200, 0.08898643256185156, 8.888771285827533, 2334900000000]
['ask', datetime.date(2021, 8, 12), '192820', 135000, 0.8387081385363993, 75.37854724724706, 1373290589000, 8.148148148148149, 81481.48148148149]
['bid', datetime.date(2021, 8, 17), '298020', 775000, -0.03915424647829759, 9.705010201181995, 2722111978000]
['bid', datetime.date(2021, 8, 18), '009540', 116500, 0.04109529595556327, 6.693871155320129, 6864992252000]
['bid', datetime.date(2021, 8, 18), '006650', 224000, 0.038586842813163165, 9.798239405517881, 1394250000000]
['bid', datetime.date(2021, 8, 18), '001800', 15850, 0.015590200445434299, 9.162996997423448, 1011723565300]
['bid', datetime.date(2021, 8, 19), '002380', 293500, -0.056705755918702784, 8.6265503889958

# 최적해 찾기
## 고정
start = "20200101"
end = "20211001"

## z_score
- 고정
position_bid = 0.8
position_ask = 0.2
z_score_period = 180
mfi_bid = 80
mfi_ask = 20
market = 'ALL'
maxPnl = 100

- 변인
z_scores= [-1.0, -1.3, -1.5, -1.7, -2.0]

|z_score|winRate|pnl|
|:-:|:-:|:-:|
|-1.0|34.32944606413994|nan|
|-1.3|35.88850174216028|283.44426401901296|
|-1.5|37.93103448275862|125.16153472630523|
|-1.7|22.22222222222222|-25.09276709655087|
|-2.0|0.0|-4.414514432628258|

## position_bid
- 고정
position_ask = 0.2
z_score = -1.4
z_score_period = 180
mfi_bid = 80
mfi_ask = 20
market = 'ALL'
maxPnl = 100

- 변인
position_bids = [0.7, 0.8, 0.9]

|position_bid|winRate|pnl|
|:-:|:-:|:-:|
|0.7|39.89637305699482|440.6378994058454|
|0.8|39.130434782608695|315.4140939709663|
|0.9|41.52542372881356|226.7342555923805|

## position_ask
- 고정
position_bid = 0.7
z_score = -1.4
z_score_period = 180
mfi_bid = 80
mfi_ask = 20
market = 'ALL'
maxPnl = 100

- 변인
position_asks = [0.1, 0.2, 0.3]

|position_ask|winRate|pnl|
|:-:|:-:|:-:|
|0.1|40.64171122994652|236.61480444712032|
|0.2|39.89637305699482|440.6378994058454|
|0.3|37.43589743589744|451.9511616901119|

## mfi_bid
- 고정
position_bid = 0.7
position_ask = 0.2
z_score = -1.4
z_score_period = 180
mfi_ask = 20
market = 'ALL'
maxPnl = 100

- 변인
mfi_bids = [70, 80, 90]

|mfi_bid|winRate|pnl|
|:-:|:-:|:-:|
|70|34.66666666666667|582.5065627636856|
|75|36.41456582633053|640.9291524639638
|76|35.625|501.1409719166681|
|77|37.06293706293706|511.48579372629183|
|78|38.82352941176471|505.1362648020545|
|79|38.32599118942731|413.3744817126745|
|80|39.89637305699482|440.6378994058454|
|90|29.411764705882355|-30.240865153798886|

## mfi_ask
- 고정
position_bid = 0.7
position_ask = 0.2
z_score = -1.4
z_score_period = 180
mfi_bid = 75
market = 'ALL'
maxPnl = 100

- 변인
mfi_asks = [10, 20, 30]

|mfi_ask|winRate|pnl|
|:-:|:-:|:-:|
|10|36.33802816901409|653.5229985590347|
|20|36.41456582633053|640.9291524639638|
|30|36.87150837988827|621.8542290878482|

## maxPnl
- 고정
position_bid = 0.7
position_ask = 0.2
z_score = -1.4
z_score_period = 180
mfi_bid = 75
mfi_ask = 20
market = 'ALL'

- 변인
maxPnls = [20, 30, 40, 100]

|maxPnl|winRate|pnl|
|:-:|:-:|:-:|
|20|36.87150837988827|953.4103012677884|
|25|37.19512195121951|867.1855323757495|
|30|36.69467787114846|906.3355600527316|
|40|36.41456582633053|691.4075004426513|
|100|36.41456582633053|640.9291524639638|

# 파라미터 최적화

In [63]:
import itertools

param_grid = {
              'market': ['KOSPI', 'KOSDAQ'],
              'position_bids': [0.6, 0.7],
              'position_asks': [0.3, 0.4],
              'mfi_bid': [75],
              'mfi_ask': [10],
              'z_scores': [-1.2, -1.3, -1.4],
              'maxPnls': [20, 30],
              'mfi_periods': [10, 14]
              }


# generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]


In [65]:
all_params[0]['market']

'KOSPI'